**Import necessary libraries**

In [30]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

**Filter warnings**

In [31]:
import warnings
warnings.filterwarnings('ignore')

**Read the Dataset**

In [32]:
data = pd.read_csv('/kaggle/input/best-books-10k-multi-genre-data/goodreads_data.csv')

**Data exploration and Preprocessing**

In [33]:
data.head()

,Unnamed: 0,Book,Author,Description,Genres,Avg_Rating,Num_Ratings,URL
0,0,To Kill a Mockingbird,Harper Lee,The unforgettable novel of a childhood in a sl...,"['Classics', 'Fiction', 'Historical Fiction', ...",4.27,"5,691,311",https://www.goodreads.com/book/show/2657.To_Ki...
1,1,Harry Potter and the Philosopher’s Stone (Harr...,J.K. Rowling,Harry Potter thinks he is an ordinary boy - un...,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...",4.47,"9,278,135",https://www.goodreads.com/book/show/72193.Harr...
2,2,Pride and Prejudice,Jane Austen,"Since its immediate success in 1813, Pride and...","['Classics', 'Fiction', 'Romance', 'Historical...",4.28,"3,944,155",https://www.goodreads.com/book/show/1885.Pride...
3,3,The Diary of a Young Girl,Anne Frank,Discovered in the attic in which she spent the...,"['Classics', 'Nonfiction', 'History', 'Biograp...",4.18,"3,488,438",https://www.goodreads.com/book/show/48855.The_...
4,4,Animal Farm,George Orwell,Librarian's note: There is an Alternate Cover ...,"['Classics', 'Fiction', 'Dystopia', 'Fantasy',...",3.98,"3,575,172",https://www.goodreads.com/book/show/170448.Ani...


In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   10000 non-null  int64  
 1   Book         10000 non-null  object 
 2   Author       10000 non-null  object 
 3   Description  9923 non-null   object 
 4   Genres       10000 non-null  object 
 5   Avg_Rating   10000 non-null  float64
 6   Num_Ratings  10000 non-null  object 
 7   URL          10000 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 625.1+ KB


In [35]:
data["Description"].fillna('', inplace=True)

In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   10000 non-null  int64  
 1   Book         10000 non-null  object 
 2   Author       10000 non-null  object 
 3   Description  10000 non-null  object 
 4   Genres       10000 non-null  object 
 5   Avg_Rating   10000 non-null  float64
 6   Num_Ratings  10000 non-null  object 
 7   URL          10000 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 625.1+ KB


**Clean the 'Genres' column (remove square brackets and quotes)**

In [37]:
data['Genres'] = data['Genres'].str.replace(r"[\[\]']", '', regex=True)

**Combine description and Genres for better recomendations**

In [40]:
data['combined_text'] = data['Description'] + ' ' + data['Genres']

**Vectorize the text using TF-IDF**

In [41]:
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
tfidf_matrix = vectorizer.fit_transform(data['combined_text'])

**Function to get recommendations based on user input**

In [42]:
def get_book_recommendations(user_input, top_n=5):
    user_tfidf = vectorizer.transform([user_input])
    similarities = cosine_similarity(user_tfidf, tfidf_matrix).flatten()
    top_indices = similarities.argsort()[-top_n:][::-1] 
    recommendations = data.iloc[top_indices][['Book', 'Author', 'Avg_Rating']].copy()
    recommendations['Similarity'] = similarities[top_indices]
    return recommendations

**Prompt the user to enter their input**

In [46]:
print("Hello! Provide us with the decription of the you are searching...")
print("\n--------------------------------------------------------------------------------\n")
while True:
    user_input = input("Few words about the book: ").strip()

    if user_input.lower() == 'exit':
        print("\nHave a nice day, Goodbye!")
        break

    if not user_input:
        print("Please enter at least one word about the book.\n")
        continue

    top_books = get_book_recommendations(user_input, top_n=5)

    print("\nTop 5 Book Recommendations:")
    print(top_books.to_string(index=False))
    print("\n--------------------------------------------------------------------------------\n")

Hello! Provide us with the decription of the you are searching...

--------------------------------------------------------------------------------



Few words about the book:  politics history fiction



Top 5 Book Recommendations:
                                                          Book              Author  Avg_Rating  Similarity
                                                 Rights of Man        Thomas Paine        4.02    0.447504
Failed States: The Abuse of Power and the Assault on Democracy        Noam Chomsky        3.98    0.419554
                                      The State and Revolution      Vladimir Lenin        4.22    0.417744
            Capital: A Critique of Political Economy, Volume 2           Karl Marx        4.08    0.403148
                                                    Discourses Niccolò Machiavelli        4.07    0.401977

--------------------------------------------------------------------------------



Few words about the book:  forgiveness



Top 5 Book Recommendations:
                                                                             Book             Author  Avg_Rating  Similarity
                                 Amish Grace: How Forgiveness Transcended Tragedy Donald B. Kraybill        3.79    0.451718
                                                                  Waiting to Live      Asa Don Brown        4.35    0.372504
Cheaper Than Therapy: How To Forgive and Overcome Anger, Anxiety, Fear and Stress     Mykey Robinson        4.35    0.363889
                                                                   My Name is Joe     Stefan Bourque        3.64    0.356230
                                                                     Resurrection        Leo Tolstoy        4.16    0.340377

--------------------------------------------------------------------------------



Few words about the book:  exit



Have a nice day, Goodbye!
